<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-some-random-data" data-toc-modified-id="Generate-some-random-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate some random data</a></span><ul class="toc-item"><li><span><a href="#Zygote-Jacobian" data-toc-modified-id="Zygote-Jacobian-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Zygote Jacobian</a></span></li><li><span><a href="#Manual-Jacobian" data-toc-modified-id="Manual-Jacobian-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Manual Jacobian</a></span></li><li><span><a href="#Comparison" data-toc-modified-id="Comparison-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Comparison</a></span></li></ul></li><li><span><a href="#Run-a-loop-and-store-both-the-error-and-the-runtime" data-toc-modified-id="Run-a-loop-and-store-both-the-error-and-the-runtime-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run a loop and store both the error and the runtime</a></span></li></ul></div>

In [ ]:
using Pkg; Pkg.activate()

using Plots
using Convex, ECOS  # Convex modeling and solver
using Distributions, Random  # Seeds and sampling
using LightGraphs  # Generating nice random graphs
using GraphPlot, Colors  # For plotting
using SparseArrays

OPT = () -> ECOS.Optimizer(verbose=false)

In [ ]:
using Revise
using CarbonNetworks

In [ ]:
using Zygote

In [ ]:
using LinearAlgebra

# Generate some random data

In [ ]:
n = 30
l = 50;
T = 5;

In [ ]:
η_c = .99;
η_d = .82;

In [ ]:
A, B, cq_dyn, cl_dyn, d_dyn, gmax_dyn, pmax_dyn, P, C = generate_random_data(n, l, T);

In [ ]:
dnet = DynamicPowerNetwork(
    cq_dyn, cl_dyn, pmax_dyn, gmax_dyn, A, B, P, C, T; η_c=η_c, η_d=η_d
)
dmin = DynamicPowerManagementProblem(dnet, d_dyn);

In [ ]:
solve!(dmin, OPT, verbose=true);

In [ ]:
x = flatten_variables_dyn(dmin);

## Zygote Jacobian

In [ ]:
_, ∂K_xT = Zygote.forward_jacobian(x -> kkt_dyn(x, dnet, d_dyn), x);

In [ ]:
_jac = sparse(∂K_xT');

## Manual Jacobian

In [ ]:
Jac_manual = compute_jacobian_kkt_dyn(x, dnet, d_dyn);

## Comparison

In [ ]:
@show norm(_jac - Jac_manual)

# Run a loop and store both the error and the runtime

In [ ]:
η_c = .99;
η_d = .82;

T = 20;

In [ ]:
n_vec = [4, 10, 50, 100];
norms = zeros(size(n_vec));
time_Zygote = zeros(size(n_vec));
time_manual = zeros(size(n_vec));

for i = 1:length(n_vec)
    n = n_vec[i]
    l = n

    A, B, cq_dyn, cl_dyn, d_dyn, gmax_dyn, pmax_dyn, P, C = generate_data(n, l, T);

    dnet = DynamicPowerNetwork(
        cq_dyn, cl_dyn, pmax_dyn, gmax_dyn, A, B, P, C, T; η_c=η_c, η_d=η_d
    )
    dmin = DynamicPowerManagementProblem(dnet, d_dyn);

    solve!(dmin, OPT, verbose=true);

    x = flatten_variables_dyn(dmin);

    tz = @elapsed _, ∂K_xT = Zygote.forward_jacobian(x -> kkt_dyn(x, dnet, d_dyn), x);
    tm = @elapsed Jac_manual = compute_jacobian_kkt_dyn(x, dnet, d_dyn);
    
    time_Zygote[i] = tz;
    time_manual[i] = tm;

    _jac = sparse(∂K_xT');

    norms[i] = norm(_jac - Jac_manual);
end

In [ ]:
plot(n_vec, norms, ls = :dash, shape=:circle)
title!("Norm of the residual")
xlabel!("n")
ylabel!("residual")

In [ ]:
plot(n_vec, time_Zygote, ls=:dash, shape=:circle, label="Zygote")
plot!(n_vec, time_manual, ls=:dash, shape=:circle, label="Manual")
title!("Computation time")
xlabel!("n")
ylabel!("Time [s]")

In [ ]:
plot(n_vec, time_Zygote./time_manual, ls=:dash, shape=:circle, label="Zygote/Manual")
title!("Ratio of computation time")
xlabel!("n")
ylabel!("Speed-up [-]")